# Imports

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

# Functions

In [ ]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [ ]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [ ]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [ ]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [ ]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

# Data Base

In [ ]:
db1 =pd.read_csv('/content/covid-19-sample.csv') 

In [ ]:
clean_data()

In [ ]:
db1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82764 entries, 0 to 82763
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               82764 non-null  datetime64[ns]
 1   autonomous_region  82764 non-null  object        
 2   province           82764 non-null  object        
 3   sex                82764 non-null  object        
 4   age_interval       82764 non-null  object        
 5   num_infections     82764 non-null  float64       
 6   num_hosp           82764 non-null  float64       
 7   num_uci            82764 non-null  float64       
 8   num_dead           82764 non-null  float64       
dtypes: datetime64[ns](1), float64(4), object(4)
memory usage: 5.7+ MB


In [ ]:
bydate = db1.groupby('date').sum().reset_index()
bydate = cumulative(bydate)
bydate = mov_7_ave(bydate)

In [ ]:
bydate.head()

,date,num_infections,num_hosp,num_uci,num_dead,cumu_num_infections,cumu_num_hosp,cumu_num_uci,cumu_num_dead,ave_7_num_infections,ave_7_num_hosp,ave_7_num_uci,ave_7_num_dead
0,2020-02-12,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2020-02-21,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-02-22,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2020-02-26,1.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2020-02-27,5.0,1.0,0.0,0.0,9.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN


# ARMA

In [ ]:
from statsmodels.tsa.arima_model import ARMA

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from sklearn.metrics import mean_squared_error as mse

# num_infections

In [50]:
infections = bydate[['date', 'ave_7_num_infections']].set_index('date')

In [56]:
infections = infections[6:]

In [57]:
infections

,ave_7_num_infections
date,
2020-03-06,11.285714
2020-03-07,15.285714
2020-03-08,18.571429
2020-03-09,27.000000
2020-03-10,43.428571
...,...
2021-06-06,1207.142857
2021-06-07,1180.142857
2021-06-08,1146.428571


In [69]:
train, test = infections[:-1], infections[-1:]

In [70]:
train

,ave_7_num_infections
date,
2020-03-06,11.285714
2020-03-07,15.285714
2020-03-08,18.571429
2020-03-09,27.000000
2020-03-10,43.428571
...,...
2021-06-05,1207.142857
2021-06-06,1207.142857
2021-06-07,1180.142857


In [71]:
test

,ave_7_num_infections
date,
2021-06-10,1055.142857


In [72]:
model=ARMA(train, order = (1, 1)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [73]:
model.predict(len(train), len(infections)-1)

2021-06-10    1073.400074
Freq: D, dtype: float64

In [74]:
test

,ave_7_num_infections
date,
2021-06-10,1055.142857


In [94]:
res=[]
for p in range(1, 5):
  for q in range(1, 10):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [110]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best


[(1, 9, 0.006666068344429732)]

# num_hosp

In [111]:
bydate.head()

,date,num_infections,num_hosp,num_uci,num_dead,cumu_num_infections,cumu_num_hosp,cumu_num_uci,cumu_num_dead,ave_7_num_infections,ave_7_num_hosp,ave_7_num_uci,ave_7_num_dead
0,2020-02-12,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,2020-02-21,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,2020-02-22,1.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,2020-02-26,1.0,1.0,0.0,0.0,4.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
4,2020-02-27,5.0,1.0,0.0,0.0,9.0,2.0,0.0,0.0,NaN,NaN,NaN,NaN


In [112]:
hosp = bydate[['date', 'ave_7_num_hosp']].set_index('date')

In [113]:
hosp = hosp[6:]

In [114]:
train, test = hosp[:-1], hosp[-1:]

In [123]:
test

,ave_7_num_hosp
date,
2021-06-10,34.428571


In [127]:
model=ARMA(train, order = (1, 2)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [125]:
model.predict(len(train), len(hosp)-1)

2021-06-10    32.340352
Freq: D, dtype: float64

In [128]:
res=[]
for p in range(1, 5):
  for q in range(1, 2):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(hosp)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/

In [129]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(3, 1, 0.05993187506358311)]

# num_uci

In [130]:
uci = bydate[['date', 'ave_7_num_uci']].set_index('date')

In [131]:
uci = uci[6:]

In [132]:
train, test = uci[:-1], uci[-1:]

In [133]:
test

,ave_7_num_uci
date,
2021-06-10,3.714286


In [143]:
model=ARMA(train, order = (1, 5)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)


In [144]:
model.predict(len(train), len(uci)-1)

2021-06-10    3.642109
Freq: D, dtype: float64

In [145]:
res=[]
for p in range(1, 5):
  for q in range(1, 5):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(uci)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [146]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 4, 0.0033038022111191445)]

# num_dead

In [147]:
dead = bydate[['date', 'ave_7_num_dead']].set_index('date')

In [148]:
dead = dead[6:]

In [149]:
train, test = dead[:-1], dead[-1:]

In [150]:
test

,ave_7_num_dead
date,
2021-06-10,3.428571


In [154]:
model=ARMA(train, order = (1, 3)).fit(disp=False)

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [155]:
model.predict(len(train), len(dead)-1)

2021-06-10    4.007331
Freq: D, dtype: float64

In [161]:
res=[]
for p in range(1, 5):
  for q in range(1, 4):
    model=ARMA(train, order = (p, q)).fit(disp=False)
    res.append((p, q, mse(model.predict(len(train), len(uci)-1), test)**0.5))

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packag

In [162]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(2, 3, 0.10383019947139394)]